<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/Clock_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as integrate
 
 # degradation rates for nuclear proteins and nuclear protein complexes
    
dx1=0.08  # degradation rate for CLOCK/BMAL
dx2=0.06  # degradation rate for PER*/CRYn
dx3=0.09  # degradation rate for PER/CRYn
dx5=0.17  # degradation rate for REV-ERB
dx6=0.12  # degradation rate for RORn
dx7=0.15  # degradation rate for BMALn
 

 # degradation rates for mRNAs 
   
dy1=0.3   # degradation rate for Per
dy2=0.2   # degradation rate for Cry
dy3=2     # degradation rate for Rev-Erb
dy4=0.2   # degradation rate for Ror
dy5=1.6   # degradation rate for Bmal


  # degradation rates for cytoplasmic proteins and cytoplasmic protein complexes 
  
dz1=0.23   # degradation rate for CRY 
dz2=0.25   # degradation rate for PER 
dz3=0.6    # degradation rate for PER*
dz4=0.2    # degradation rate for PER*/CRYc
dz5=0.2    # degradation rate for PER/CRYc
dz6=0.31   # degradation rate for REV-ERB
dz7=0.3    # degradation rate for RORc
dz8=0.73   # degradation rate for BMALc 


  # reaction rates for complex formation/dissociation
  
kfx1=2.3   # complex formation rate for CLOCK/BMAL
kdx1=0.01  # complex dissociation rate for CLOCK/BMAL

kfz4=1     # complex formation rate for PER*/CRY
kfz5=1     # complex formation rate for PER/CRY

kdz5=1     # complex dissociation rate for PER/CRY
kdz4=1     # complex dissociation rate for PER*/CRY


  # phosphorylation rates 

kfz2=2      # phosphorylation rate constant for PER  
kdfz3=0.05  # dephosphorylation rate constant for PER*


  # transcription rates
  
V1max=1      # maximal rate of Per transcription
V2max=2.92   # maximal rate of Cry transcription
V3max=1.90   # maximal rate of Rev-Erb transcription
V4max=10.90  # maximal rate of Ror transcription
V5max=1      # maximal rate of Bmal transcription

kt1=3     # rate constant of Per transcription
kt2=2.4   # rate constant of Cry transcription
kt3=2.07  # rate constant of Rev-Erb transcription
kt4=0.9   # rate constant of Ror transcription
kt5=8.35  # rate constant of Bmal transcription

ki1=0.9   # inhibition constant of Per transcription
ki2=0.7   # inhibition constant of Cry transcription
ki21=5.2  # inhibition constant of Cry transcription by REV-ERB
ki3=3.3   # inhibition constant of Rev-Erb transcription
ki4=0.4   # inhibition constant of Ror transcription
ki5=1.94  # inhibition constant of Bmal transcription


   # translation rates
    
kp1=0.4     # production rate of PER  
kp2=0.26    # production rate of CRY
kp3=0.37    # production rate of REV-ERB 
kp4=0.76    # production rate of ROR
kp5=1.21    # production rate of BMAL

   # import/export rates
    
kiz4=0.2   # import rate of  PER*/CRY to the nucleus
kiz5=0.1   # import rate of  PER/CRY to the nucleus
kiz6=0.5   # import rate of REV-ERB to the nucleus
kiz7=0.1   # import rate of ROR to the nucleus
kiz8=0.1   # import rate of BMAL to the nucleus

kex2=0.02  # export rate of  PER*/CRY to the cytoplasm
kex3=0.01  # export rate of  PER/CRY to the cytoplasm


   # fold activation of transcription
 
a=12  # fold activation of Per transcription
d=12  # fold activation of Cry transcription
g=5   # fold activation of Rev-Erb transcription 
h=5   # fold activation of Ror transcription
i=12  # fold activation of Bmal transcription


  # Hill coefficients of transcription

b=5    # Hill coefficient of activation of Per transcription
c=7    # Hill coefficient of inhibition of Per transcription

e=6    # Hill coefficient of activation of Cry transcription
f=4    # Hill coefficient of inhibition of Cry transcription
f1=1   # Hill coefficient of inhibition of Cry transcription via REV

v=6    # Hill coefficient of activation of Rev-Erb  transcription
w=2    # Hill coefficient of inhibition of Rev-Erb  transcription

p=6    # Hill coefficient of activation of Ror transcription
q=3    # Hill coefficient of inhibition of Ror transcription

n=2    # Hill coefficient of activation of Bmal transcription
m=5    # Hill coefficient of inhibition of Bmal transcription


a2=1  # weight factor for PER*/CRY
a3=1  # weight factor for PER/CRY

Rev0=0   # simulating a constitutively expressed Rev-Erb with the given value
Ror0=0   # simulating a constitutively expressed Ror with the given value
Bmal0=0  # simulating a constitutively expressed Bmal with the given value
Per0=0   # simulating a constitutively expressed Per with the given value
Cry0=0   # simulating a constitutively expressed Cry with the given value

 
dt = 0.01
t = np.arange(0.0, 50.0, dt) #The time to integrate over 
len_t = len(t) 
timestep = 20 #timestep
 
timecount = 0 #global

In [13]:

#ODEs

def dALLdt(X, t):

   global timecount
   
   PC, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19 = X

   PC = a2 * x18 + a3 * x19 # pool of nuclear complexes PER/CRY

   x1 = (kfx1 * x10 - kdx1 * x1 - dx1 * x1)   # CLOCK/BMAL

   x2 = (V3max * (1 + g * ((x1 / kt3) ^ v)) / (1 + ((PC / ki3) ^ w) * ((x1 / kt3) ^ v) + (x1 / kt3) ^ v ) - dy3 * x2)   # transcription of Rev-erb

   x3 = (V4max * (1 + h * ((x1 / kt4) ^ p)) / (1 + ((PC / ki4) ^ q) * ((x1 / kt4) ^ p) + (x1 / kt4) ^ p ) - dy4 * x3);   # transcription of Ror

   x4 = (kp3 *(x2 + Rev0) - kiz6 * x4 - dz6 * x4)   # production of REV-ERB 

   x5 = (kp4 *(x3 + Ror0) - kiz7 * x5 - dz7 * x5)   # production of ROR

   x6 = (kiz6 * x4 - dx5 * x6)   # nuclear REV-ERB 

   x7 = (kiz7 * x5 - dx6 * x7)   # nuclear ROR

   x8 = (V5max * (1 + i * ((x7 / kt5) ^ n)) / (1 + (x6 / ki5) ^ m + (x7 / kt5) ^ n ) - dy5 * x8) # transcription of Bmal

   x9 = (kp5 * (x8 + Bmal0) - kiz8 * x9 - dz8 * x9)   # production of BMAL

   x10 = (kiz8 * x9 + kdx1 * x1 -  kfx1 * x10 - dx7 * x10)   # nuclear BMAL

   x11 = (V1max * (1 + a * ((x1 / kt1) ^ b))  / (1 + ((PC / ki1) ^ c) * ((x1 / kt1 ) ^ b)  + (x1 / kt1 ) ^ b) - dy1 * x11) # transcription of Per

   x12 = ((V2max * (1 + d * ((x1 / kt2) ^ e))  / (1 + ((PC / ki2) ^ f) * ((x1 / kt2 ) ^ e)  + (x1 / kt2 ) ^ e)) * (1 / (1 + (x6 / ki21) ^ f1)) - dy2 * x12) # transcription of Cry

   x13 = (kp2 * (x12 + Cry0)  + kdz4 * x16 + kdz5 * x17 - kfz5 * x13 * x14 - kfz4 * x13 * x15 - dz1 * x13) # production of CRY 

   x14 = (kp1 * (x11 + Per0)  + kdz5 * x17  + kdfz3 * x15 - kfz5 * x14 * x13 - kfz2 * x14 - dz2 * x14) # production of PER 

   x15 = (kfz2 * x14 + kdz4 * x16 - kdfz3 * x15 - kfz4 * x15 * x13  - dz3 * x15) # phosphorylation of PER 

   x16 = (kfz4 * x13 * x15 + kex2 * x18 - kiz4 * x16 - kdz4 * x16 - dz4 * x16)  # complex formation-PER*/CRY -cytoplasm

   x17 = (kfz5 * x13 * x14 + kex3 * x19 - kiz5 * x17 - kdz5 * x17 - dz5 * x17) # complex formation-PER/CRY -cytoplasm

   x18 = (kiz4 * x16 - kex2 * x18 - dx2 * x18) # nuclear complex-PER*/CRY 

   x19 = (kiz5 * x17 - kex3 * x19 - dx3 * x19) # nuclear complex-PER/CRY 

   return PC, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19
  
   X0 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
   X= integrate.odeint(dALLdt, X0, t)


   V = X[:,0]
   m = X[:,1]
   h = X[:,2]
   n = X[:,3]
   plt.plot(V, t)


     


In [ ]:
 #微分方程式を解く
    X= integrate.odeint(dALLdt, X0, t)

    V = X[:,0]
    m = X[:,1]
    h = X[:,2]
    n = X[:,3]
   

IndentationError: ignored

In [ ]:
#timestep後のyの値を次のステップでのy0の値に更新する
    X0 = (V[timestep], m[timestep], h[timestep], n[timestep]) 
     
    #描画
    ax1.set_title('Hodgkin-Huxley Neuron')
    ax1.plot(t, V, 'k')
    ax1.plot(t[len_t-1],V[len_t-1],'ko')
    ax1.set_ylabel('V (mV)')
    ax1.set_ylim([-80,50])
    ax2.grid()
     
    ax2.plot(t, ina, 'c', label='$I_{Na}$')
    ax2.plot(t, ik, 'y', label='$I_{K}$')
    ax2.plot(t, il, 'm', label='$I_{L}$')
    ax2.plot(t[len_t-1],ina[len_t-1],'co')
    ax2.plot(t[len_t-1],ik[len_t-1],'yo')
    ax2.plot(t[len_t-1],il[len_t-1],'mo')
    ax2.set_ylabel('Current')
    ax2.set_ylim([-900,900])
    ax2.grid()
    ax2.legend(bbox_to_anchor=(0, 1),
           loc='upper left',
           borderaxespad=0)
     
    ax3.plot(t, m, 'r', label='m')
    ax3.plot(t, h, 'g', label='h')
    ax3.plot(t, n, 'b', label='n')
    ax3.plot(t[len_t-1],m[len_t-1],'ro')
    ax3.plot(t[len_t-1],h[len_t-1],'go')
    ax3.plot(t[len_t-1],n[len_t-1],'bo')
    ax3.set_ylabel('Gating Value')
    ax3.legend(bbox_to_anchor=(0, 1),
               loc='upper left',
               borderaxespad=0)
     
    i_inj_values = [I_inj(t,timecount) for t in t]
    ax4.plot(t, i_inj_values, 'k')
    ax4.plot(t[len_t-1], i_inj_values[len_t-1],'ko')
    ax4.set_xlabel('t (ms)')
    ax4.set_ylabel('$I_{inj}$ ($\\mu{A}/cm^2$)')
    ax4.set_ylim(-2, 40)
 
ani = animation.FuncAnimation(fig, update, interval=100,
                              frames=100)
plt.show() #表示